<a href="https://colab.research.google.com/github/nxxk23/AI-Engineer/blob/main/fraud/iclaim.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip -q install langchain langchain_huggingface langchain_community  gradio transformers elasticsearch gradio_calendar

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.1/57.1 MB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.1/320.1 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 543.1/543.1 kB 22.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 35.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.4/64.4 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.1/11.1 MB 54.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 41.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.8/63.8 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 

In [2]:
import os
from getpass import getpass
import requests
import os
import json
from huggingface_hub import InferenceClient
from elasticsearch import Elasticsearch
import pandas as pd
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn.functional as F
from sentence_transformers import SentenceTransformer
import gradio as gr
import json
from gradio_calendar import Calendar

In [3]:
# Elasticsearch credentials
user = "natthaphol.po"
passwords = "cnEM5CeFrG"
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')
model = AutoModel.from_pretrained('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')
es = Elasticsearch(
    hosts=[{'host': 'elk.manageai.co.th', 'port': 443, 'scheme': 'https'}],
    basic_auth=(user, passwords)
)

def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

def rag_encode(text):
    encoded_input = tokenizer(text, padding=True, truncation=True, return_tensors='pt')
    with torch.no_grad():
        model_output = model(**encoded_input)
    sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])
    sentence_embeddings = F.normalize(sentence_embeddings, p=2, dim=1)
    return sentence_embeddings.squeeze().tolist()

def search_exact_match_score(query_text, index_name):
    query_vector = rag_encode(query_text)
    search_query = {
        "size": 10,
        "_source": ["icd10", "total_average_price"],  # Include only required fields
        "query": {
            "script_score": {
                "query": {
                    "match_all": {}
                },
                "script": {
                    "source": "cosineSimilarity(params.query_vector, 'embedding') + 1.0",
                    "params": {
                        "query_vector": query_vector
                    }
                }
            }
        }
    }

    try:
        response = es.search(index=index_name, body=search_query, timeout='120s')
        exact_score_matches = [
            {
                "_source": hit["_source"]
            }
            for hit in response['hits']['hits'] if hit["_score"] == 2.0
        ]

        if exact_score_matches:
            return exact_score_matches
        else:
            print("No documents found with score 2.0.")
            return None

    except Exception as e:
        print(f"Error performing search: {e}")
        return None

def search_similar_vectors(query_text, index_name):
    query_vector = rag_encode(query_text)

    search_query = {
        "size": 1000,
        "_source": [
            "simb_billing_code", "local_billing_name", "item_id", "item_name",
            "average_initial_price", "order_count", "icd10", "range"
        ],
        "query": {
            "script_score": {
                "query": {
                    "match_all": {}
                },
                "script": {
                    "source": "cosineSimilarity(params.query_vector, 'embedding') + 1.0",
                    "params": {
                        "query_vector": query_vector
                    }
                }
            }
        }
    }

    try:
        response = es.search(index=index_name, body=search_query, timeout='120s')
        results = []
        for hit in response['hits']['hits']:
            score = hit["_score"]
            if score == 2.0:
                result = {
                    "_index": hit["_index"],
                    "_id": hit["_id"],
                    "_score": score,
                    "_source": hit["_source"]
                }
                results.append(result)

        if results:
            source_data = [hit["_source"] for hit in results]
            df = pd.DataFrame(source_data)
            return df
        else:
            print("No results with score 2.0 found.")
            return None

    except Exception as e:
        print(f"Error performing search: {e}")
        return None

tokenizer_config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/645 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/471M [00:00<?, ?B/s]

In [9]:
# กำหนดตัวแปร Global สำหรับเก็บข้อมูลที่อัปโหลด
uploaded_data = None
uploaded_simb = None
uploaded_diag = None
sample_cids = []
diagnosis_mapping = {}

# ฟังก์ชันที่โหลดไฟล์และดึงข้อมูล sample_cids
def load_files(data_file, simb_file, diag_file):
    global uploaded_data, uploaded_simb, uploaded_diag, sample_cids, diagnosis_mapping

    try:
        # โหลดไฟล์ CSV
        if data_file:
            uploaded_data = pd.read_csv(data_file.name)
        if simb_file:
            uploaded_simb = pd.read_csv(simb_file.name)
        if diag_file:
            uploaded_diag = pd.read_csv(diag_file.name)

        # สร้าง diagnosis_mapping จาก diag_file
        if uploaded_diag is not None:
            diagnosis_mapping = {
                row['ICD-10 Code']: row['Description'].strip()
                for _, row in uploaded_diag.iterrows()
            }
        else:
            diagnosis_mapping = {}

        # ตรวจสอบว่า data_file มีข้อมูล
        if uploaded_data is not None:
            # ดึง sample_cids ที่ไม่ใช่การติดตาม (is_follow_up == 'N')
            sample_cids = uploaded_data[uploaded_data['is_follow_up'] == 'N']['cid'].sample(5).tolist()
            print(f"Sample CIDs: {sample_cids}")  # ตรวจสอบค่าของ sample_cids
        else:
            sample_cids = []

        # คืนค่าผลลัพธ์และรายการ CID ตัวอย่าง
        return "ไฟล์ถูกอัปโหลดสำเร็จ!", gr.update(choices=sample_cids), gr.update(choices=list(diagnosis_mapping.values()))

    except Exception as e:
        # คืนค่าข้อความแสดงข้อผิดพลาด
        return f"เกิดข้อผิดพลาดในการอัปโหลดไฟล์: {str(e)}", gr.update(choices=[]), gr.update(choices=[])

# ฟังก์ชันสำหรับใช้งานไฟล์ที่อัปโหลด
def analyze_uploaded_data():
    global uploaded_data, uploaded_simb, uploaded_diag

    if uploaded_data is None or uploaded_simb is None or uploaded_diag is None:
        return "กรุณาอัปโหลดไฟล์ทั้งหมดก่อน", None

    # ตัวอย่างการประมวลผลข้อมูล (เช่น นับจำนวนแถวในแต่ละไฟล์)
    data_summary = {
        "data.csv": len(uploaded_data),
        "simblist.csv": len(uploaded_simb),
        "diaglist.csv": len(uploaded_diag),
    }
    result_message = "ประมวลผลไฟล์สำเร็จ!"
    return result_message, data_summary

def update_diagnosis_choices(cid):
    global diagnosis_mapping
    patient_row = uploaded_data[uploaded_data["cid"] == cid]
    if not patient_row.empty:
        icd10_code = patient_row['icd10'].iloc[0]
        diagnosis_description = diagnosis_mapping.get(icd10_code, "")
        return diagnosis_description  # Set as single choice and default value
    return ""

# Mappings for illness type and follow-up status
illness_type_mapping = {
    "ILL": "เจ็บป่วย",
    "ACC": "อุบัติเหตุ",
    "FU": "ผู้ป่วยนอกติดตามอาการหลังจาก Admit เจ็บป่วย",
    "ER": "อุบัติเหตุฉุกเฉินภายใน 24 ชม.",
    "HD": "ผู้ป่วยนอกล้างไต",
    "DEN": "ทันตกรรม"
}

follow_up_mapping = {
    "N": "เข้ารับการรักษาครั้งแรก",
    "Y": "เข้ารักษาแบบต่อเนื่อง"
}

def populate_patient_info(cid):
    global uploaded_data, uploaded_diag
    # ค้นหาผู้ป่วยตาม CID
    patient_row = uploaded_data[uploaded_data["cid"] == cid]
    if not patient_row.empty:
        patient_data = json.loads(patient_row.iloc[0]["patient"])  # Parse JSON
        icd10_code = patient_row.iloc[0]["icd10"]
        diagnosis_description = uploaded_diag[uploaded_diag['ICD-10 Code'] == icd10_code]['Description'].iloc[0] if not uploaded_diag[uploaded_diag['ICD-10 Code'] == icd10_code].empty else ""
        search_result = search_exact_match_score(icd10_code, 'internal-manageai-icd10-average_price')
        average_price = search_result[0]['_source']['total_average_price'] if search_result else 0.0

        # ตัวอย่าง: การแปลง Mapping
        illness_type_code = patient_row["illnes_type"].iloc[0]
        illness_type_mapped = illness_type_mapping.get(illness_type_code, "อุบัติเหตุ")
        visit_type_mapped = follow_up_mapping.get(patient_row["is_follow_up"].iloc[0], "Unknown")
        chief_complaint = patient_row["chief_complaint"].iloc[0]
        billing_table, total_net = get_billing_table_with_order_items(cid)

        # ตรวจสอบว่าค่าของ illness_type ถูกส่งไปที่ฟิลด์ที่เหมาะสม
        return (
            patient_data.get("title_name", ""),
            patient_data.get("first_name", ""),
            patient_data.get("last_name", ""),
            patient_data.get("hn", ""),
            patient_data.get("dob", ""),
            illness_type_mapped,  # ส่งไปที่ช่อง TextBox ของ treatment_type
            visit_type_mapped,
            chief_complaint,
            diagnosis_description.strip(),
            average_price,
            billing_table.values.tolist(),
            total_net,
            cid
        )
    else:
        return "", "", "", "", "", "", "", "", "", 0.0, [], [], ""

def update_diagnosis_choices(cid):
    patient_row = uploaded_data[uploaded_data["cid"] == cid]
    if not patient_row.empty:
        icd10_code = patient_row['icd10'].iloc[0]
        diagnosis_description = diagnosis_mapping.get(icd10_code, "")
        return diagnosis_description  # Set as single choice and default value
    return ""

def get_simb_code(bill_description):
    description_to_simb = dict(zip(simb['Description'], simb['SIMB_1']))
    return description_to_simb.get(bill_description, "")

def get_billing_table_with_order_items(cid):
    patient_row = uploaded_data[uploaded_data["cid"] == cid]
    if not patient_row.empty:
        billing_data = json.loads(patient_row.iloc[0]["billing"])
        order_items_data = json.loads(patient_row.iloc[0]["order_item"])

        billing_df = pd.DataFrame(billing_data)
        order_items_df = pd.DataFrame(order_items_data)

        all_rows = []

        for _, billing_row in billing_df.iterrows():
            matching_items = order_items_df[order_items_df['simb_billing_code'] == billing_row['simb_billing_code']]

            # Add a row for the billing category
            all_rows.append({
                "ลำดับ": len(all_rows) + 1,
                "หมวดหมู่ค่าบริการ": f"{billing_row['simb_billing_code']} {billing_row['local_billing_name']}",
                "รายการ": "",
                "จำนวน": billing_row.get('item_amount', ""),  # Use get to safely access the key
                "จำนวนเงิน(ก่อนหักส่วนลด)": float(billing_row['billing_initial']),
                "ส่วนลด": float(billing_row['billing_discount']),
                "จำนวนเงิน(หลังหักส่วนลด)": float(billing_row['billing_net_amount']),
                "Billing Code": "",
                "SIMB": billing_row['simb_billing_code']
            })

            # Add rows for each item under this category
            for _, item in matching_items.iterrows():
                all_rows.append({
                    "ลำดับ": "",
                    "หมวดหมู่ค่าบริการ": item['item_id'],
                    "รายการ": item['item_name'],
                    "จำนวน": item['item_amount'],  # Access item_amount from order_items_df
                    "จำนวนเงิน(ก่อนหักส่วนลด)": float(item['initial']),
                    "ส่วนลด": float(item['discount']),
                    "จำนวนเงิน(หลังหักส่วนลด)": float(item['net_amount']),
                    "Billing Code": billing_row['local_billing_code'],
                    "SIMB": billing_row['simb_billing_code']
                })

        # Summation of numeric columns
        total_initial = billing_df['billing_initial'].astype(float).sum()
        total_discount = billing_df['billing_discount'].astype(float).sum()
        total_net = billing_df['billing_net_amount'].astype(float).sum()

        all_rows.append({
            "ลำดับ": "",
            "หมวดหมู่ค่าบริการ": "รวม",
            "รายการ": "",
            "จำนวน": "",
            "จำนวนเงิน(ก่อนหักส่วนลด)": total_initial,
            "ส่วนลด": total_discount,
            "จำนวนเงิน(หลังหักส่วนลด)": total_net,
            "Billing Code": "",
            "SIMB": ""
        })

        # Grand total row
        all_rows.append({
            "ลำดับ": "",
            "หมวดหมู่ค่าบริการ": "รวมค่ารักษาพยาบาลทั้งหมด",
            "รายการ": "",
            "จำนวน": "",
            "จำนวนเงิน(ก่อนหักส่วนลด)": total_initial,
            "ส่วนลด": total_discount,
            "จำนวนเงิน(หลังหักส่วนลด)": total_net,
            "Billing Code": "",
            "SIMB": ""
        })

        return pd.DataFrame(all_rows), total_net  # Return both DataFrame and total_net

    return pd.DataFrame(columns=["ลำดับ", "หมวดหมู่ค่าบริการ", "รายการ", "จำนวน", "จำนวนเงิน(ก่อนหักส่วนลด)", "ส่วนลด", "จำนวนเงิน(หลังหักส่วนลด)", "Billing Code", "SIMB"]), 0.0

def update_billing_table_with_order_items(cid):
    billing_df, total_net = get_billing_table_with_order_items(cid)
    return billing_df.values.tolist(), total_net

def calculate_recommended_claim(total_net, remaining_opd_year, total_average_price):
    total_net = float(total_net) if isinstance(total_net, str) else total_net
    remaining_opd_year = float(remaining_opd_year) if isinstance(remaining_opd_year, str) else remaining_opd_year
    total_average_price = float(total_average_price) if isinstance(total_average_price, str) else total_average_price
    if remaining_opd_year >= total_average_price:
        recommended_claim = total_average_price - total_net
    else:
        recommended_claim = remaining_opd_year - total_net

    return recommended_claim

def recommend_drugs_for_patient(patient_icd10_code):
    output = search_similar_vectors(patient_icd10_code, 'internal-manageai-icd10-order')
    if output is not None:
        filtered_output = output[output['simb_billing_code'] == '1.1.1(3)']
        sorted_output = filtered_output.sort_values(by=['order_count'], ascending=False)
        top = sorted_output.drop(columns=['order_count','icd10','range']).head(10)
        return top
    else:
        return "No results found for the given ICD10 code."

def handle_form(
    id_card, title, first_name, last_name, patient_id, birth_date, remaining_opd_year,
    treatment_type, visit_type, chief_complaint, diag_type, diagnosis, total_net
):
    total_net_float = float(total_net)
    remaining_opd_year_float = float(remaining_opd_year) if remaining_opd_year else 0.0
    icd10_code = diagnosis.split(" ")[0] if diagnosis else None

    # Calculate recommended claim
    output = search_exact_match_score(icd10_code, 'internal-manageai-icd10-average_price')
    if output:
        average_price = float(output[0]['_source'].get('total_average_price', 0.0))
        max_claimable = max(0, average_price - total_net_float)
        additional_claim = min(max_claimable, remaining_opd_year_float - total_net_float) if max_claimable > 0 else 0.0
        recommendation_adjustment = f"{'+ ' if additional_claim >= 0 else '- '}{abs(additional_claim):.2f}"
        recommended_claim = total_net_float + additional_claim
        claim_message = f"{recommended_claim:.2f}"
    else:
        recommendation_adjustment = "ไม่สามารถคำนวณได้"
        claim_message = "ไม่มีข้อมูลราคากลางสำหรับการรักษานี้"

    # Search recommended drugs for patient
    recommended_drugs = recommend_drugs_for_patient(icd10_code)
    if isinstance(recommended_drugs, pd.DataFrame):
        recommended_drugs_list = recommended_drugs.values.tolist()
    else:
        recommended_drugs_list = []  # Empty if no results

    return recommendation_adjustment, claim_message, recommended_drugs_list

custom_css = """
.red-border {
    border: 2px solid red !important;
    border-radius: 5px;
    padding: 5px;
}
"""
### Ui
with gr.Blocks() as demo:
    with gr.Tab("Upload Data Files"):
        gr.Markdown("### อัปโหลดไฟล์ CSV ที่ใช้ในการวิเคราะห์")

        # Input for CSV files
        with gr.Row():
            data_file = gr.File(label="อัปโหลดไฟล์ data.csv")
            simb_file = gr.File(label="อัปโหลดไฟล์ simblist.csv")
            diag_file = gr.File(label="อัปโหลดไฟล์ diaglist.csv")

        # Display message after uploading files
        upload_button = gr.Button("อัปโหลดไฟล์")
        upload_result = gr.Textbox(label="ผลลัพธ์การอัปโหลดไฟล์", interactive=False)

    with gr.Tab("Patient and Claim Information"):
        gr.Markdown("## ข้อมูลผู้ป่วยนอก")
        id_card = gr.Textbox(label="เลขบัตรประชาชน *", placeholder="กรอกเลขบัตรประชาชนของผู้ป่วย", elem_classes="red-border")
        sample_cid_buttons = gr.Radio(label="เลือกตัวอย่าง CID", choices=[], type="value")
        with gr.Row():
            title = gr.Dropdown(
                label="คำนำหน้า",
                choices=["คุณ", "นาง", "นางสาว", "นาย", "Mr", "Ms", "Miss", "Mrs", "เด็กชาย", "เด็กหญิง"]
            )
            first_name = gr.Textbox(label="ชื่อ")
            last_name = gr.Textbox(label="นามสกุล")
        with gr.Row():
            patient_id = gr.Textbox(label="รหัสผู้ป่วย (HN)")
            birth_date = Calendar(label="วัน/เดือน/ปีเกิด (ค.ศ.)")

        gr.Markdown("### ข้อมูลส่งเคลม")

        with gr.Row():
            treatment_type = gr.Dropdown(
                label="ประเภทการรักษา",
                value="",  # ค่าที่ได้จากการแมพ
                choices=list(illness_type_mapping.values())
)
            visit_type = gr.Radio(
                label="ประเภทการเข้ารักษา",
                choices=list(follow_up_mapping.values())
            )
        chief_complaint = gr.Textbox(label="อาการสำคัญที่เข้ามาโรงพยาบาล", value="")

        gr.Markdown("#### Diagnosis Type")
        with gr.Row():
            diag_type = gr.Dropdown(
                label="Diagnosis Type",
                choices=["โรคหลักที่ให้การรักษา (PP)", "โรคประจำตัว (UD)", "การวินิจฉัยร่วม (CM)", "โรคแทรก (CP)", "โรคอื่นๆ (OT)"],
                value='โรคหลักที่ให้การรักษา (PP)'
            )
            diagnosis = gr.Dropdown(
                label="Diagnosis (ICD10) *",
                choices=list(diagnosis_mapping.values()) if diagnosis_mapping else [],
                value=""
            )

        gr.Markdown("#### รายการค่ารักษาพยาบาล")
        billing_table = gr.DataFrame(
            headers=["ลำดับ", "หมวดหมู่ค่าบริการ", "รายการ", "จำนวน", "จำนวนเงิน(ก่อนหักส่วนลด)", "ส่วนลด", "จำนวนเงิน(หลังหักส่วนลด)", "Billing Code", "SIMB"],
            interactive=True
        )

    with gr.Tab("Recommend Claim"):
        gr.Markdown("#### จำนวนเงินที่สามารถเบิกได้")
        with gr.Row():
            total_net = gr.Number(label="รวมค่ารักษาพยาบาลในการส่งยอดเคลมครั้งนี้ (บาท) ", interactive=False)
            average_price = gr.Number(label="จำนวนเบิกได้สูงสุดสำหรับโรคนี้ (บาท)", interactive=False)

        id_card.change(
            fn=update_billing_table_with_order_items,
            inputs=[id_card],
            outputs=[billing_table, total_net]
        )

        remaining_opd_year = gr.Textbox(label="จำนวนเงิน OPD คงเหลือ *", placeholder="กรอกจำนวนเงิน OPD คงเหลือ", elem_classes="red-border")

        with gr.Row(elem_classes="red-border"):
            recommendation_adjustment = gr.Textbox(label="แนะนำให้เบิกเพิ่ม(+)/ลด(-) (บาท)", interactive=False, visible=True)
            final_result = gr.Textbox(label="สามารถแนะนำให้เบิกได้สูงสุดสำหรับครั้งนี้ (บาท)", interactive=False, visible=True)  # เพิ่มช่องนี้

        id_card.change(
            fn=populate_patient_info,
            inputs=[id_card],
            outputs=[title, first_name, last_name, patient_id, birth_date, treatment_type, visit_type, chief_complaint, diagnosis, average_price, billing_table, total_net, id_card]
        )

        id_card.change(
            fn=update_diagnosis_choices,
            inputs=[id_card],
            outputs=[diagnosis]
        )

        sample_cid_buttons.change(
            fn=populate_patient_info,
            inputs=[sample_cid_buttons],
            outputs=[title, first_name, last_name, patient_id, birth_date, treatment_type, visit_type, chief_complaint, diagnosis, average_price, billing_table, total_net, id_card]
        )

        upload_button.click(
            fn=load_files,
            inputs=[data_file, simb_file, diag_file],
            outputs=[upload_result, sample_cid_buttons, diagnosis]
        )

        gr.Markdown("##### รายการยาที่มักใช้ในผู้ป่วยโรคเดียวกัน")
        recommended_drugs_table = gr.DataFrame(
            headers=["SIMB", "Billing Name", "Item ID", "Item Name", "Average Price"], interactive=False)

        submit_button = gr.Button("Submit")
        submit_button.click(
            fn=handle_form,
            inputs=[id_card, title, first_name, last_name, patient_id, birth_date, remaining_opd_year,
                    treatment_type, visit_type, chief_complaint, diag_type, diagnosis, total_net],
            outputs=[recommendation_adjustment, final_result, recommended_drugs_table]
        )

        remaining_opd_year.submit(
            fn=handle_form,
            inputs=[id_card, title, first_name, last_name, patient_id, birth_date, remaining_opd_year,
                    treatment_type, visit_type, chief_complaint, diag_type, diagnosis, total_net],
            outputs=[recommendation_adjustment, final_result, recommended_drugs_table]
        )

/usr/local/lib/python3.10/dist-packages/gradio/components/dropdown.py:226: UserWarning: The value passed into gr.Dropdown() is not in the list of choices. Please update the list of choices to include:  or set allow_custom_value=True.
  warnings.warn(


In [10]:
demo.launch(debug=True, share=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://4f9a88b037aef34a36.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Sample CIDs: ['1129902423661', '1349900464384', '3102300162061', '1102002870231', '3101500979311']


<ipython-input-3-9dcbbe866668>:45: DeprecationWarning: Received 'timeout' via a specific parameter in the presence of a 'body' parameter, which is deprecated and will be removed in a future version. Instead, use only 'body' or only specific parameters.
  response = es.search(index=index_name, body=search_query, timeout='120s')
<ipython-input-3-9dcbbe866668>:45: DeprecationWarning: Received 'timeout' via a specific parameter in the presence of a 'body' parameter, which is deprecated and will be removed in a future version. Instead, use only 'body' or only specific parameters.
  response = es.search(index=index_name, body=search_query, timeout='120s')
<ipython-input-3-9dcbbe866668>:45: DeprecationWarning: Received 'timeout' via a specific parameter in the presence of a 'body' parameter, which is deprecated and will be removed in a future version. Instead, use only 'body' or only specific parameters.
  response = es.search(index=index_name, body=search_query, timeout='120s')
<ipython-inpu

Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7861 <> https://4f9a88b037aef34a36.gradio.live
